# function_io

> I/O functions used for loading / saving results and local variables from function.

In [ ]:
#| default_exp core.function_io

In [ ]:
#| export
import pdb
import joblib
import os
import re
import argparse
import shlex
from dataclasses import dataclass
from functools import reduce
from pathlib import Path
import sys
import ast
import logging
import warnings

from IPython import get_ipython
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)
from IPython.core.magic_arguments import (argument, magic_arguments, parse_argstring)
import ipynbname
from sklearn.utils import Bunch
from fastcore.all import argnames
import nbdev

## df

In [ ]:
#| export
def load_df (path, **kwargs):
    if (path.parent / f'{path.name}.parquet').exists():
        df = pd.read_parquet (path.parent / f'{path.name}.parquet', **kwargs)
    elif (path.parent / f'{path.name}.pk').exists():
        df = pd.read_parquet (path.parent / f'{path.name}.pk', **kwargs)
    elif (path.parent / f'{path.name}.csv').exists():
        df = pd.read_parquet (path.parent / f'{path.name}.csv', **kwargs)
    else:
        raise RuntimeError (f'File {path} not found')
    return df

In [ ]:
#| export
def save_df (df, path, **kwargs):
    path.parent.mkdir (parents=True, exist_ok=True)
    name_without_extension = path.name[:-len('.df')]
    extension = ''
    try:
        df.to_parquet (path.parent / f'{name_without_extension}.parquet', **kwargs)
        extension='parquet'
    except:
        try:
            df.to_pickle (path.parent / f'{name_without_extension}.pk', **kwargs)
            extension='pickle'
        except:
            df.to_csv (path.parent / f'{name_without_extension}.csv', **kwargs)
            extension='csv'
    with open (path, 'wt') as f: 
        f.write (extension)

## pickle

In [ ]:
#| export
def load_pickle (path, **kwargs):
    return joblib.load (path, **kwargs)

In [ ]:
#| export
def save_pickle (data, path, **kwargs):
    joblib.dump (data, path, **kwargs)

## csv

In [ ]:
#| export
def load_csv (path, **kwargs):
    return pd.read_csv (path, **kwargs)

In [ ]:
#| export
def save_csv (df, path, **kwargs):
    df.to_csv (path, **kwargs)

## parquet

In [ ]:
#| export
def load_parquet (df, **kwargs):
    return pd.read_parquet (path, **kwargs)

In [ ]:
#| export
def save_parquet (df, path, **kwargs):
    df.to_parquet (path, **kwargs)

## load

In [ ]:
#| export
def load (
    path_variables,
    io_type='pickle',
    **kwargs,
):
    load_function = eval (f'load_{io_type}')
    return load_function (path_variables, **kwargs)

## save

In [ ]:
#| export
def save (
    data,
    path_variables,
    io_type='pickle',
    **kwargs,
):
    Path(path_variables).parent.mkdir (parents=True, exist_ok=True)
    save_function = eval (f'save_{io_type}')
    return save_function (data, path_variables, **kwargs)